In [7]:
import pandas as pd
import torch
from torch import nn
from transformers import DistilBertTokenizer, DistilBertModel
from torch.utils.data import DataLoader,DataSet

def saveTensors():
    """
        Save the text ans tensors so we don't need to do this everytime
    """
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    df = pd.read_csv("train.csv")
    torch.save( [ [ tokenizer( txt , return_tensors="pt")  for txt in df.text.values ], torch.LongTensor( df.target.values ) ], 'train.data' )
    
# saveTensors()



ImportError: cannot import name 'DataSet'

In [ ]:
# Create Model
class TweetClf(nn.Module):
    
    def __init__(self):
        super(TweetClf,self).__init__()
        

In [8]:
class TweetDataSet(DataSet):
    
    def __init__(self):
        super( TweetDataSet, self ).__init__()
        self.data = torch.load('train.data')
        
    def __getitem__(self,index):
        return self.data[index]
    def __len__(self):return len( self.data )

NameError: name 'DataSet' is not defined

In [5]:
# Create Variables
dataset = DataLoader( TweetDataSet(),batch_size=BATCH_SIZE )

NameError: name 'DataLoader' is not defined